<h1>Add Machine Learning Model CSV results to HEROKU Database</h1>
<br>
TEAMS PREDICTIONS

<h4>Import Dependencies</h4>

In [1]:
import os
import csv
import pandas as pd
import random

import sqlite3
import csv
from sqlalchemy import create_engine

from config import pgPassword

<h4>Create paths to CSV files</h4>

In [2]:
pathTeams = os.path.join("..", "..", "data", "csv/models", "teamPredictions.csv")
#pathRunsScore = os.path.join("..", "..", "data", "csv/models", "rpgPredictions.csv")
#pathHomeRuns = os.path.join("..", "..", "data", "csv/models", "HRpgPredictions.csv")
#pathBaseHits = os.path.join("..", "..", "data", "csv/models", "hpgPredictions.csv")
#pathWalks = os.path.join("..", "..", "data", "csv/models", "BBpgPredictions.csv")

<h4>Load CSV files into DataFrame</h4>

In [3]:
# Read Team Predictions csv into a DataFrame
teamPredictDF = pd.read_csv(pathTeams)
teamPredictDF = teamPredictDF.rename(columns={'Unnamed: 0': 'ID'})
teamPredictDF = teamPredictDF.rename(columns={'model_type': 'modelType'})
teamPredictDF['ID'] = teamPredictDF.index
print(teamPredictDF.shape)
teamPredictDF.head()

(4132, 6)


,ID,yearID,teamID,actual,model,modelType
0,0,1999,ANA-1999,0.432099,0.489055,ML-EN-T1
1,1,1999,ANA-1999,0.432099,0.472244,ML-LN-T1
2,2,1999,ANA-1999,0.432099,0.498932,ML-LS-T1
3,3,1999,ANA-1999,0.432099,0.472354,ML-RD-T1
4,4,2000,ANA-2000,0.506173,0.524125,ML-EN-T1


<h1>===========================================================</h1>

<h1>Set up SQL DataBase</h1>

Before running of the cells below- <b>
    in pgAdmin: drop and create above tables with blank content in DataBase named 'baseball_db'

<h4>Create a connection to SQL database</h4>

In [4]:
pg_user = 'user'
pg_password = 'password'
db_name = 'name'

connection_string = f"{pg_user}:{pg_password}@heroku.com:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

In [5]:
engine.table_names()

['Batting',
 'Franchises',
 'Team-Stats',
 'Teams',
 'FranchisePlayers',
 'Pitching',
 'Players',
 'PlayerPredictions-RunsScore',
 'PlayerPredictions-HomeRuns',
 'PlayerPredictions-Walks',
 'TeamPredictions',
 'PlayerPredictions-BaseHits',
 'Salaries']

In [6]:
teamsDF = pd.read_sql_table("Teams", con = engine)
teamsDF.head()

,teamID,TeamName
0,BS1-1871,Boston Red Stockings
1,CH1-1871,Chicago White Stockings
2,CL1-1871,Cleveland Forest Citys
3,FW1-1871,Fort Wayne Kekiongas
4,NY2-1871,New York Mutuals


In [7]:
mergeTeamsDF = pd.merge(teamPredictDF, teamsDF, on="teamID")
print(mergeTeamsDF.shape)
mergeTeamsDF.head()

(4132, 7)


,ID,yearID,teamID,actual,model,modelType,TeamName
0,0,1999,ANA-1999,0.432099,0.489055,ML-EN-T1,Anaheim Angels
1,1,1999,ANA-1999,0.432099,0.472244,ML-LN-T1,Anaheim Angels
2,2,1999,ANA-1999,0.432099,0.498932,ML-LS-T1,Anaheim Angels
3,3,1999,ANA-1999,0.432099,0.472354,ML-RD-T1,Anaheim Angels
4,4,2000,ANA-2000,0.506173,0.524125,ML-EN-T1,Anaheim Angels


In [8]:
teamPredictDF2 = mergeTeamsDF.copy()
teamPredictDF2['ID'] = teamPredictDF2.index
teamPredictDF2 = teamPredictDF2[teamPredictDF2.columns.drop('TeamName')]
print(teamPredictDF2.shape)
teamPredictDF2.head()

(4132, 6)


,ID,yearID,teamID,actual,model,modelType
0,0,1999,ANA-1999,0.432099,0.489055,ML-EN-T1
1,1,1999,ANA-1999,0.432099,0.472244,ML-LN-T1
2,2,1999,ANA-1999,0.432099,0.498932,ML-LS-T1
3,3,1999,ANA-1999,0.432099,0.472354,ML-RD-T1
4,4,2000,ANA-2000,0.506173,0.524125,ML-EN-T1


<h4>Add dataframes to SQL database</h4>

In [9]:
teamPredictDF2.to_sql(name = "TeamPredictions", con = engine, if_exists='append', index=False)
print("team predictions loaded success...")

team predictions loaded success...
